In [ ]:
!pip install yfinance
!pip install numpy
!pip install matplotlib
!pip install mplfinance
!pip install pandas

In [ ]:
import yfinance as yf
from matplotlib import pyplot as plt
from datetime import datetime, timedelta
from darts import TimeSeries
import pandas as pd
import requests
import itertools
from darts.dataprocessing.transformers import Scaler
from darts.utils.statistics import plot_acf, check_seasonality

In [ ]:
today = datetime.today().date()
data = pd.DataFrame()
cryptos = ['BTC', 'ETH', 'USDT', 'BNB', 'SOL', 'ADA', 'DOGE']

In [ ]:
for crypto in cryptos:
    asset = yf.download(f'{crypto}-USD', interval= '1d', end=today)
    if crypto == 'BTC':
        data.index = asset.index
        ind = list(pd.date_range(min(data.index), today))
        data = data.reindex(index=ind)
    data[f'{crypto}'] = asset['Close']

In [ ]:
# Forward fill missing values
data = data.fillna(method="ffill")
# Drop today's date for which we don't have yet the close price yet
data = data.loc[data.index!=str(today)]
data = data.fillna(0)

In [ ]:
# we are dealing with a multivariate series where each column corresponds to a different cryptocurrency, the values represent crypto closing prices, and the indices consist of dates
data = data.reset_index()
series = TimeSeries.from_dataframe(data, time_col="Date", value_cols=cryptos)
# Create Training & Validation Series
val_set_size = 28
val_set_split_date = max(data["Date"])-timedelta(days=val_set_size)
train, val = series.split_before(val_set_split_date)
train.plot(label='training')
val.plot(label='validation')

In [ ]:
# Scaling these series to a range of 0 to 1
scaler = Scaler()
scaler = scaler.fit(train)
train_scaled = scaler.transform(train)
train_scaled = train_scaled.astype("float32")
val_scaled = scaler.transform(val)
val_scaled = val_scaled.astype("float32")
train_scaled.plot()